In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
from slack import SlackInteractor
from claude_llm import ClaudeLLM
from project_manager_agent import ProjectManagerAgent
from sarcastic_agent import SarcasticAgent
from paul_graham_agent import PaulGrahamAgent  # Add this import
from db import ActionDatabase
from runner import process_threads, execute_due_actions

In [46]:
slack_interactor = SlackInteractor()
llm = ClaudeLLM()
action_db = ActionDatabase()

agents = [
    ProjectManagerAgent(llm, action_db, slack_interactor),
    SarcasticAgent(llm, action_db, slack_interactor),
    PaulGrahamAgent(llm, action_db, slack_interactor)  # Add the new agent here
]

In [47]:
print("\nFetching new messages...")
data = slack_interactor.fetch_new_messages()
threads = slack_interactor.organize_threads(data)
print(f"Found {len(threads)} threads with new messages.")

results = process_threads(agents, threads)

print("\nChecking for due actions...")
execute_due_actions(agents)


Fetching new messages...
Set conversations_oldest to 1722399936.894109
Loaded 449 old messages
Found 0 new messages
Updated complete_conversations.pkl with new messages
Found 0 threads with new messages.

Checking for due actions...

Executing delayed action for Paul Graham in thread: 2024-08-28 21:31:09.097969055
Action: Post a startup or tech-related joke in the channel once per day
Rate limited. Retrying in 1.81 seconds (attempt 1/10)
Rate limited. Retrying in 2.78 seconds (attempt 2/10)
ClaudeLLM prompt:

        You are an AI agent with the following characteristics:
        Personality: Insightful, direct, and focused on startups and technology
        Goal: Provide thought-provoking insights on startups, technology, and innovation. Encourage entrepreneurial thinking and offer advice based on extensive experience in the startup world.
        Username: Paul Graham
Due task to execute: Post a startup or tech-related joke in the channel once per day

        Analyze the following 